In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [106]:
def fix_JSON(json_message=None):
    result = None
    try:        
        result = json.loads(json_message)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(e.message.split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        json_message = list(json_message)
        json_message[idx_to_replace] = ' '
        new_message = ''.join(json_message)     
        return fix_JSON(json_message=new_message)
    return result

In [105]:
explainers = BeautifulSoup(requests.get("http://theconversation.com/au/factcheck").text)
headlines = explainers.find_all('h2')
results = list()

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [148]:
for headline in headlines:
    thisline = dict()
    try:
        link = "http://theconversation.com" + headline.a['href']
        thisline['headline'] = headline.text
        thisline['content'] = requests.get(link).text
        soup = BeautifulSoup(thisline['content'])
        thisline['link'] = link
        meta = soup.find(attrs={'name' : "news_keywords"})
        thisline['keywords'] = meta['content']
    except:
        pass
    results.append(thisline)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [52]:
abcexplainers = BeautifulSoup(requests.get('http://www.abc.net.au/news/factcheck/zombies/').text)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [116]:
links = abcexplainers.find_all("a")
links = [link for link in links if "Fact Check zombie" in link.text]

In [ ]:
thisline = list()

In [119]:
link = links[0]

In [ ]:
abcid = link['href'].split('/')[-1]
url = "https://content-api-govhack.abc-prod.net.au/v1/" + abcid
data = requests.get(url).text

In [142]:
jsondoc = json.loads(data)

JSONDecodeError: Expecting ',' delimiter: line 1 column 708 (char 707)

In [ ]:
thisline.append(jsondoc)

In [118]:
for link in links:
    abcid = link['href'].split('/')[-1]
    url = "https://content-api-govhack.abc-prod.net.au/v1/" + abcid
    data = requests.get(url).text
    jsondoc = json.loads(data.replace('\r\n', ''))
    thisline.append(jsondoc)

JSONDecodeError: Invalid \escape: line 73 column 499 (char 9310)

In [101]:
df = pd.DataFrame(results)

In [102]:
df

,content,headline,keywords,link
0,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Election FactCheck: Does the government spend ...,NaN,http://theconversation.com/election-factcheck-...
1,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Has the Coalition invested an average of $5 bi...,NaN,http://theconversation.com/election-factcheck-...
2,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Has the Coalition cut bulk-billing for patholo...,NaN,http://theconversation.com/election-factcheck-...
3,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,"Have 300,000 new jobs been created in the last...",NaN,http://theconversation.com/election-factcheck-...
4,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Is the Australian Sex Party right about religi...,NaN,http://theconversation.com/election-factcheck-...
5,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Does the government spend more on negative gea...,NaN,http://theconversation.com/election-factcheck-...
6,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,"Are larger, more frequent storms predicted due...",NaN,http://theconversation.com/election-factcheck-...
7,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,"Have 50,000 full-time jobs been lost this year...",NaN,http://theconversation.com/election-factcheck-...
8,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Is it true no solely managed Commonwealth fish...,NaN,http://theconversation.com/election-factcheck-...
9,<!DOCTYPE html>\n<!--[if IE 7]> <html class...,Has $100 billion been added to Australia’s nat...,NaN,http://theconversation.com/election-factcheck-...


In [18]:
df.to_pickle('explainers.pickle')